Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", remove any ```raise NotImplementedError()``` and enter your student ID below:

In [1]:
STUDENT_ID = "200774408"

---

# Coursework 8

This is the solution notebook for coursework 8 of module MTH793P at Queen Mary University of London.

Author: [Martin Benning](mailto:m.benning@qmul.ac.uk)

Date: 30.03.2020

Last updated: 18.03.2021

First we load the necessary libraries.

In [2]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

If you need to import additional libraries, please do this in the following cell and do not attempt to overwrite the previous cell as this will lead to a failure of the autograder system.

In [3]:
from scipy.linalg import svd

Next, we use the provided routines that allow us to load the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) and store the rankings in a matrix.

In [4]:
def read_txt(path):
    """read text file from path."""
    with open(path, "r") as path:
        return path.read().splitlines()

def load_data(dataset_path):
    """Load data in text format, one rating per line, as in the kaggle competition."""
    data = read_txt(dataset_path)[1:]
    return preprocess_data(data)

def preprocess_data(data):
    """preprocessing the text data, conversion to numerical array format."""
    def process_line(line):
        position, rating = line.split(',')
        row, column = position.split("_")
        row = row.replace("r", "")
        column = column.replace("c", "")
        return int(row), int(column), float(rating)

    def statistics(data):
        row = set([line[0] for line in data])
        column = set([line[1] for line in data])
        return min(row), max(row), min(column), max(column)

    # parse each line
    data = [process_line(line) for line in data]

    # perform basic statistics on the dataset.
    min_row, max_row, min_column, max_column = statistics(data)
    print("number of items: {}, number of users: {}".format(max_row, max_column))

    # build rating matrix.
    ratings = sp.lil_matrix((max_row, max_column))
    for row, column, rating in data:
        ratings[row - 1, column - 1] = rating
    return ratings

We load and store the data via *load_data*. Please use the path at which you have stored the csv file *movielens100k.csv*.

In [5]:
ratings = load_data('movielens100k.csv').todense()
dimensions = ratings.shape

number of items: 1682, number of users: 943


This $1682 \times 943$ matrix has movie ratings for 1682 movies from 943 users in the range of 1 to 5. Missing ratings are set to zero.

In [6]:
print("The first ten ratings for the first ten items of the first ten users are \n{r}.".format(r = \
        ratings[0:10, 0:10].astype(int)))

The first ten ratings for the first ten items of the first ten users are 
[[0 4 0 0 4 4 0 0 0 4]
 [3 0 0 0 3 0 0 0 0 0]
 [4 0 0 0 0 0 0 0 0 0]
 [3 0 0 0 0 0 5 0 0 4]
 [3 0 0 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0 5 0]
 [4 0 0 0 0 2 5 3 4 4]
 [1 0 0 0 0 4 5 0 0 0]
 [5 0 0 0 0 4 5 0 0 4]
 [3 2 0 0 0 0 4 0 0 0]].


Complete the following four functions. For the functions **soft_shrinkage** and **nuclear_norm_proximal_mapping** you can use your implementations from Coursework 7. The function **projection** takes two arguments *matrix* and *known_indices*, and returns a (one-dimensional) array of known entries, which are the entries of *matrix* specified at the indices *known_indices*. If *matrix* is a $n \times s$ matrix, then the values of *known_indices* range from 0 to $ns - 1$.

The function **transpose_projection** takes three arguments *known_entries*, *known_indices* and *matrix_dimensions*, and computes the transpose of the operation **projection**, i.e. if $P_\Omega:\mathbb{R}^{n \times s} \rightarrow \mathbb{R}^r$ is the mathematical notation of the operator that performs **projection**, then **transpose_projection** should satisfy

$$ \sum_{l = 1}^r (P_\Omega M)_j v_j = \langle P_\Omega M, v \rangle = \langle M, P_\Omega^\top v \rangle = \sum_{i = 1}^n \sum_{j = 1}^s M_{ij} (P^\top v)_{ij} \, , $$

for all matrices $M \in \mathbb{R}^{n \times s}$ and vectors $\mathbb{R}^r$.

In [7]:
def projection(matrix, known_indices):
    ## Maybe?
    nrows, ncols = matrix.shape
    rows = known_indices // ncols
    cols = known_indices % ncols
    return matrix[rows, cols].T

def transpose_projection(known_entries, known_indices, matrix_dimensions):
    n_entries = np.prod(matrix_dimensions)
    proj = np.zeros(n_entries)
    proj[known_indices] = np.array(known_entries).ravel()
    proj = proj.reshape(matrix_dimensions)
    return proj

def soft_shrinkage(input_array, regularisation_parameter=1):
    return np.sign(input_array) * np.maximum(np.abs(input_array) - regularisation_parameter, 0)

def nuclear_norm_proximal_mapping(matrix, regularisation_parameter=1):
    u, s, vh = svd(matrix, full_matrices=False)
    shat = np.maximum(s - regularisation_parameter, 0)
    return (u * shat[..., None, :]) @ vh

We test **transpose_projection** with the following cell that is worth **1/5 marks**.

In [8]:
from numpy.testing import assert_array_equal
known_test_indices = np.array([2, 3, 5, 7, 8, 9])
known_test_ratings = np.array([-4, 3, 1, -2, -1, 4])
T = transpose_projection(known_test_ratings, known_test_indices, (4, 3))
assert_array_equal(T, np.array([[0, 0, -4], [3, 0, 1], [0, -2, -1], [4, 0, 0]]))

Next, we store all indices for which the entries in the matrix *ratings* are non-zero in a one-dimensional array *known_indices*. We then store the corresponding entries of *ratings* in an array *known_ratings*, for which we use our function **projection**. Subsequently, we subtract the value three from *known_ratings* and store it in a variable named *known_ratings_centred*.

In [9]:
_, known_indices = np.where(ratings.ravel() != 0)
known_ratings = projection(ratings, known_indices)
known_ratings_centred = known_ratings - 3

We write a function **matrix_completion** that is based on the linearised Bregman iteration and accelerated linearised Bregman iteration as introduced in the lecture notes, for fixed step-size $\tau = 1$. The function takes the arguments as specified in the function header. Here, *known_indices* are the indices at which the entries of the matrix are known, while *known_entries* are the corresponding entries and *matrix_dimensions* the dimension of the output matrix. The complete *matrix* is initilised as a zero matrix of correct size. The argument *scaling_parameter* corresponds to the balancing parameter in front of the nuclear norm ($\gamma$ in the lecture notes). The parameter *maximum_no_of_iterations* specifies the maximum number of iterations before the algorithm terminates, while *tolerance* is a quantity that forces the iteration to stop when

$$ \frac12 \| P_\Omega L^k - z \|^2_{\text{Fro}} \leq \text{tolerance} $$

is achieved. Here $P_\Omega L^k$ is the mathematical notation of the projection applied to the matrix *L^k* at iteration $k$, and $z$ is the mathematical notation for the known entries. The parameter *acceleration* indicates if acceleration is enabled or disabled. If it is enabled, the adaptive parameter $\beta_k$ should be chosen as

$$ \beta_0 = 0 \qquad \text{and} \qquad \beta_k = \frac{k - 1}{k + 3} \, , $$

for $k = 1, 2, \ldots$. The function should return the matrix $L$ (variable name *low_rank_matrix*) as well as a list *sensitivities* of values $\frac12 \| P_\Omega L^k - z \|^2_{\text{Fro}}$ for all iterates $k = 0, 1, \ldots, K$, where $K$ denotes the final iterate.

In [10]:
def adaptive(k):
    return (k - 1) / (k + 3) * (k >= 1)

def matrix_completion(known_indices, known_entries, matrix_dimensions, scaling_parameter=10, \
                      maximum_no_iterations=1000, tolerance=10**(-8), acceleration=False, print_output=50):
    τ = 1
    known_entries = np.array(known_entries).ravel()
    yk = known_entries.copy()
    zk = known_entries.copy()
    z = known_entries.copy()
    sensitivities = []
    for k in range(maximum_no_iterations):
        PΩ = transpose_projection(yk, known_indices, matrix_dimensions)
        Lk = nuclear_norm_proximal_mapping(τ * PΩ, scaling_parameter)
        update_term = projection(Lk, known_indices) - z
        zk_next = yk - update_term
        if acceleration:
            βk = adaptive(k)
            yk = (1 + βk) * zk_next - βk * zk
            zk = zk_next
        else:
            yk = zk_next
        
        sensitivity = np.linalg.norm(update_term) ** 2 / 2
        sensitivities.append(sensitivity)
        
        if k % 5 == 0:
            end = "\n" if k % 50 == 0 else "\r"
            print(f"@{k=:04} | {sensitivity=:0.7f}", end=end)
        if sensitivity <= tolerance:
            break
        
    return Lk, sensitivities

Test your code with the following two tests (one visible, one hidden), which account for **2/5 marks**.

In [11]:
from numpy.testing import assert_array_almost_equal
test_recovered, _ = matrix_completion(known_test_indices, known_test_ratings, (4, 3), scaling_parameter=100, \
                                        tolerance=10**(-14), acceleration=True)
assert_array_almost_equal(test_recovered, [[-1.36244095, -0.94262418, -4], [3, 0.03212887, 1], \
                                           [-0.00652379, -2, -1], [4, -0.01572708, 1.10956443]])

@k=0000 | sensitivity=23.5000000
@k=0050 | sensitivity=0.0000421
@k=0100 | sensitivity=0.0000000
@k=0150 | sensitivity=0.0000000
@k=0200 | sensitivity=0.0000000
@k=0250 | sensitivity=0.0000000


Complete the ratings of your matrix ratings from the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with your matrix completion algorithm with $\gamma = 100$ and activated acceleration. Store the completed matrix for the uncentered ratings in a variable named *complete_ratings_1*.

In [13]:
%%time
complete_ratings_1, _ = matrix_completion(known_indices, known_ratings, ratings.shape,
                                       acceleration=True, scaling_parameter=100)

@k=0000 | sensitivity=352983.5598261
@k=0050 | sensitivity=0.0043527806
@k=0100 | sensitivity=0.0000152
@k=0150 | sensitivity=0.0000003
@k=0200 | sensitivity=0.0000000
CPU times: user 4min 15s, sys: 2.67 s, total: 4min 18s
Wall time: 1min 11s


Repeat the previous excerise for the centred ratings and store the completed matrix in a variable named *complete_ratings_2*. Don't forget to add the value three back to your completed, centred ratings.

In [14]:
%%time
complete_ratings_2, _ = matrix_completion(known_indices, known_ratings_centred, ratings.shape,
                                          acceleration=True, scaling_parameter=100)
complete_ratings_2 = complete_ratings_2 + 3

@k=0000 | sensitivity=70173.5853500
@k=0050 | sensitivity=0.04147127767
@k=0100 | sensitivity=0.0004230
@k=0150 | sensitivity=0.0000179
@k=0200 | sensitivity=0.0000016
@k=0250 | sensitivity=0.0000002
@k=0300 | sensitivity=0.0000000
@k=0350 | sensitivity=0.0000000
CPU times: user 7min 51s, sys: 4.96 s, total: 7min 56s
Wall time: 2min 7s


In [15]:
print("A sample of the first 8 incomplete ratings are \n\n{};\n\nthe corresponding, rounded ratings from".format(\
        ratings[0:8, 0:8].astype(int))) 
print("the uncentered date are\n\n{};\n\nthe corresponding, rounded ratings from the centred data".format(\
        np.rint(complete_ratings_1[0:8, 0:8]).astype(int)))
print("are \n\n{}.".format(np.rint(complete_ratings_2[0:8, 0:8]).astype(int)))

A sample of the first 8 incomplete ratings are 

[[0 4 0 0 4 4 0 0]
 [3 0 0 0 3 0 0 0]
 [4 0 0 0 0 0 0 0]
 [3 0 0 0 0 0 5 0]
 [3 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0]
 [4 0 0 0 0 2 5 3]
 [1 0 0 0 0 4 5 0]];

the corresponding, rounded ratings from
the uncentered date are

[[ 2  4  0  0  4  4  2  2]
 [ 3  0  0  0  3  1  2  2]
 [ 4  0  0  0  0  0  0  0]
 [ 3  0  0  0  1  2  5  2]
 [ 3  0  0  0  1  1  2  0]
 [ 5  1  0  0  0  0  1 -1]
 [ 4  1  0  1  3  2  5  3]
 [ 1  1  0  0  2  4  5  1]];

the corresponding, rounded ratings from the centred data
are 

[[3 4 3 3 4 4 4 4]
 [3 3 3 3 3 3 3 3]
 [4 3 3 3 3 3 3 3]
 [3 3 3 3 3 4 5 3]
 [3 3 3 3 3 4 3 3]
 [5 3 3 3 3 3 3 3]
 [4 3 3 3 4 2 5 3]
 [1 3 3 3 3 4 5 3]].


In [16]:
print('The rank of the non-centred matrix is %d. \n\nThe rank of the centred matrix is %d.'
      %(np.linalg.matrix_rank(complete_ratings_1), np.linalg.matrix_rank(complete_ratings_2 - 3)))

The rank of the non-centred matrix is 385. 

The rank of the centred matrix is 317.


Formulate a projected (or proximal) gradient descent algorithm for a matrix completion problem of the form
\begin{align*}
\hat L = \arg\min_{L \in \mathbb{R}^{s \times n}} \left\{ \frac12 \| P_\Omega L - z \|_{\text{Fro}}^2 \quad \text{subject to} \quad \text{rank}(L) \leq k \right\} \, .
\end{align*}
Here $P_\Omega \in \mathbb{R}^{s \times n} \rightarrow \mathbb{R}^r$ is a (known) projection operator that projects the known entries, specified by the index set $\Omega$, of its argument to a vector of length $r$. The vector of known entries is denoted by $z \in \mathbb{R}^r$ and $k \in \mathbb{N}$ is a fixed constant that determines the rank of $\hat L$. The term $\text{rank}(L)$ determines the rank of the matrix-argument $L$. What does the proximal mapping (in the proximal gradient descent) look like? What is its closed-form solution?

In [17]:
def adaptive(k):
    return (k - 1) / (k + 3) * (k >= 1)

def matrix_completion_via_projected_gradient_descent(known_indices, known_entries, matrix_dimensions, rank,
                      maximum_no_iterations=500, tolerance=10**(-8), print_output=50, acceleration=True):
    τ = 1
    scaling_parameter = 100
    known_entries = np.array(known_entries).ravel()
    yk = known_entries.copy()
    zk = known_entries.copy()
    z = known_entries.copy()
    sensitivities = []
    for k in range(maximum_no_iterations):
        PΩ = transpose_projection(yk, known_indices, matrix_dimensions)
        Lk = nuclear_norm_proximal_mapping(τ * PΩ, scaling_parameter)
        # Rank restriction
        u, s, vh = svd(Lk, full_matrices=False)
        Lk = (u[:, :rank] * s[:rank][..., None, :]) @ vh[:rank, :]
        update_term = projection(Lk, known_indices) - z
        zk_next = yk - update_term
        if acceleration:
            βk = adaptive(k)
            yk = (1 + βk) * zk_next - βk * zk
            zk = zk_next
        else:
            yk = zk_next
        
        sensitivity = np.linalg.norm(update_term) ** 2 / 2
        sensitivities.append(sensitivity)
        
        if k % 5 == 0:
            end = "\n" if k % 50 == 0 else "\r"
            print(f"@{k=:04} | {sensitivity=:0.7f}", end=end)
        if sensitivity <= tolerance:
            break
        
    return Lk, sensitivities

Test your function with the following hidden tests that are worth **2/5 marks**.

In [18]:
from numpy.testing import assert_array_almost_equal
test_recovered, _ = matrix_completion(known_test_indices, known_test_ratings, (4, 3), 3)
test_recovered

@k=0000 | sensitivity=10.8069783


array([[-0.17656506, -0.32419455, -3.99997583],
       [ 2.99997372,  0.0505304 ,  1.00007501],
       [-0.01217548, -1.9999906 , -1.00008496],
       [ 4.00001746, -0.02796713,  0.16368334]])

Complete the ratings of your matrix ratings from the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with your projected gradient descent matrix completion algorithm with *rank* = 300. Store the completed matrix for the centered ratings in a variable named *complete_ratings_3*. Don't forget to add the value three back to your completed, centred ratings.

In [18]:
complete_ratings_3, _ = matrix_completion_via_projected_gradient_descent(known_indices, known_ratings_centred,
                                                                         ratings.shape, 300)
complete_ratings_3 = complete_ratings_3 + 3

@k=0000 | sensitivity=70173.5853500
@k=0050 | sensitivity=1.44126974767
@k=0100 | sensitivity=1.2661776
@k=0150 | sensitivity=1.5022756
@k=0200 | sensitivity=1.4112130
@k=0250 | sensitivity=1.4820117
@k=0300 | sensitivity=1.1462796
@k=0350 | sensitivity=1.0810680
@k=0400 | sensitivity=1.5470282
@k=0450 | sensitivity=1.2729709


In [19]:
print("A sample of the first 8 incomplete ratings are \n\n{};\n\nthe corresponding, rounded ratings from".format(\
        ratings[0:8, 0:8].astype(int))) 
print("the uncentered date are\n\n{};\n\nthe corresponding, rounded ratings from the centred data".format(\
        np.rint(complete_ratings_1[0:8, 0:8]).astype(int)))
print("are \n\n{};\n\nthe corresponding, rounded ratings with the alternative matrix completion are\n\n{}".format(\
        np.rint(complete_ratings_2[0:8, 0:8]).astype(int), np.rint(complete_ratings_3[0:8, 0:8]).astype(int)))

A sample of the first 8 incomplete ratings are 

[[0 4 0 0 4 4 0 0]
 [3 0 0 0 3 0 0 0]
 [4 0 0 0 0 0 0 0]
 [3 0 0 0 0 0 5 0]
 [3 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0]
 [4 0 0 0 0 2 5 3]
 [1 0 0 0 0 4 5 0]];

the corresponding, rounded ratings from
the uncentered date are

[[ 2  4  0  0  4  4  2  2]
 [ 3  0  0  0  3  1  2  2]
 [ 4  0  0  0  0  0  0  0]
 [ 3  0  0  0  1  2  5  2]
 [ 3  0  0  0  1  1  2  0]
 [ 5  1  0  0  0  0  1 -1]
 [ 4  1  0  1  3  2  5  3]
 [ 1  1  0  0  2  4  5  1]];

the corresponding, rounded ratings from the centred data
are 

[[3 4 3 3 4 4 4 4]
 [3 3 3 3 3 3 3 3]
 [4 3 3 3 3 3 3 3]
 [3 3 3 3 3 4 5 3]
 [3 3 3 3 3 4 3 3]
 [5 3 3 3 3 3 3 3]
 [4 3 3 3 4 2 5 3]
 [1 3 3 3 3 4 5 3]];

the corresponding, rounded ratings with the alternative matrix completion are

[[3 4 3 3 4 4 4 4]
 [3 3 3 3 3 3 3 3]
 [4 3 3 3 3 3 3 3]
 [3 3 3 3 3 4 5 3]
 [3 3 3 3 3 4 3 3]
 [5 3 3 3 3 3 3 3]
 [4 3 3 3 4 2 5 3]
 [1 3 3 3 3 4 5 3]]


This is the end of this week's coursework.

## References

* https://arxiv.org/pdf/0909.5457.pdf
* https://www.stat.cmu.edu/~ryantibs/convexopt-S15/scribes/08-prox-grad-scribed.pdf
* https://www.stat.cmu.edu/~ryantibs/convexopt/lectures/prox-grad.pdf